In [1]:
# General Purpose
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import ode
from scipy import special, integrate
from math import erf

# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, interact_manual

%matplotlib inline

style = {'description_width': '200px'}
slider_layout = Layout(width='50%')

In [2]:
#### This function calculates the concentration as a function of time, and is only used with the ODE
#### solver used in the main function
def dispcalc_ON(time, Lt, Rt, Q, D, timeSplit):    
    
    Ut = 2*Q/3.14159/Rt**2
    k = Rt**2*Ut**2/192/D

    if time < timeSplit:
        CONC = 0.5 - 0.5*special.erf(0.5*(Lt - Ut/2*time)*k**(-0.5)*pow(time,-0.5))
    else:
        ### here we need to shift the time range to start from 0 (time - timeSplit), and also change the equation
        ### describing the dispersion (- changes to a +)
        tt = time - timeSplit + 0.000001   # small number added to get rid of zero condition
        CONC = 0.5 + 0.5*special.erf(0.5*(Lt - Ut/2*tt)*k**(-0.5)*pow(tt,-0.5))
    return CONC

In [3]:
#### This function calculates the concentration as a function of time, by looping over the array time
#### I have to do this twice because I sometimes don't know how to program efficiently
def calcCONC(time, Lt, Rt, Q, D, timeSplit):
    CONC = np.zeros(len(time))
    for i in range(len(time)):
        CONC[i] = dispcalc_ON(time[i], Lt, Rt, Q, D, timeSplit)
    return CONC
    

In [4]:
def main(C, K1, K2, Bo, timeMax, Lt, Rt, Q, D, timeSplit):
    
    
    Cr = C*1e-14 ### Cr [=] mol/mm3        C [=] nM
    K1r = K1*1e6 ### K1r[=] mm3/mol/s     K1 [=] 1/M/s
    Bor = Bo*1e-14 ### Bor [=] mol/mm2    Bo [=] ng/cm2
    
    ### Set W and H manually for now
    W = 14 ### [mm] width of the QCM flow cell
    L = 14 ### [mm] length of the QCM flow cell
    H = 0.1 ### [mm] height of the QCM flow cell
    U = Q/H/D #### [mm/s]
    KM = 1.47*(Q*D**2/L/H**2/W)**(1/3)

    def function(AB, time):
        CONC = Cr*dispcalc_ON(time, Lt, Rt, Q, D, timeSplit)
        return (CONC*K1r*(Bor - AB) - K2*AB)/(1 + K1r*Bor/KM)
     

    Kd = K2/K1r
    time = np.linspace(1, timeMax, 3600)
    
    
    ### working odeint, but gives strange results sometimes
    solution = odeint(function, 0, time, rtol = 1e-20, atol = 1e-20)
    CC = calcCONC(time, Lt, Rt, Q, D, timeSplit)
    FracMax = (Cr)/(Cr + Kd)
    
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20, 10))
    
    # Plot fractional coverage of AB vs. time - the solution to the ODE described by function()    
    ax1.plot(time, solution/Bor, 'k.', label='signal')
    ax1.set_title('Estimated signal', fontsize=20)
    ax1.set_xlabel('Time (s)', fontsize=14)
    ax1.set_ylabel('Fractional Coverage', fontsize=14) 
    if max(solution/Bor) > 0.1:
        ax1.set_ylim([0,1])    
    ax1.legend(loc='best')
    ax1.grid()
    
    
    ax2.plot(time, CC, 'r.', label = 'conc')
    ax2.set_title('Normalized Concentration', fontsize=20)
    ax2.set_xlabel('Time (s)', fontsize=14)
    ax2.set_ylabel('C/Co', fontsize=14)
    ax2.grid()
    
    plt.show()
    
    DA = K1r*Bor/KM
    print('Damkohler Number = %.1f' % DA)



In [5]:
interact(main, C = FloatSlider(min=0.01, max=100, step=0.05, value=5, description='Concentration (nM)', style=style, layout=slider_layout),
               K1 = FloatSlider(min=1000, max=1e6, step=2000, value=3e4, description='K1 (1/M/s)', style=style, layout=slider_layout, readout_format='1.2e'),
               K2 = FloatSlider(min=1e-4, max=1e-2, step=1e-4, value=1e-3, description='K2 (1/s)', style=style, layout=slider_layout, readout_format='.3e'),
               Bo = FloatSlider(min=0.05, max=10, step=0.5, value=1, description='Bo (ng/cm2)', style=style, layout=slider_layout),
               timeMax = IntSlider(min=100, max=3600, step=5, value=3600, description='MaxTime', style=style, layout=slider_layout),
               Lt = FloatSlider(min=10, max = 1000, step = 10, value = 200, description='Inlet tubing length (mm)', style=style, layout=slider_layout),
               Rt = FloatSlider(min=0.02, max = 0.5, step = 0.01, value = 0.29, description='Tubing inner radius (mm)', style=style, layout=slider_layout),
               Q = FloatSlider(min=0.03, max = 1.7, step = 0.03, value = 0.33, description='Flow rate (mm3/s)', style=style, layout=slider_layout),
               D = FloatSlider(min=1e-6, max = 1e-4, step = 1e-6, value = 4e-5, description='Analyte Diffusivity (mm2/s)', style=style, layout=slider_layout, readout_format='.2e'),
               timeSplit = FloatSlider(min=10, max = 3600, step = 1, value = 1800, description='Split time (s)', style=style, layout=slider_layout),
        );

interactive(children=(FloatSlider(value=5.0, description='Concentration (nM)', layout=Layout(width='50%'), min…